In [2]:
import os
import cv2
import numpy as np
import math

In [4]:
cap = cv2.VideoCapture('umcp.mpg')
ret,frame = cap.read()
img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
w,h = img.shape

wc1=np.zeros((w,h,3),dtype=np.float64,order='C')    #wc1 = prior probability of Gaussian i for c1 (blue)
mc1=np.zeros((w,h,3),dtype=np.float64,order='C')    #mc1 = mean vector [r b g]T of pixel (x,y)     
wc2=np.zeros((w,h,3),dtype=np.float64,order='C')
mc2=np.zeros((w,h,3),dtype=np.float64,order='C')
wc3=np.zeros((w,h,3),dtype=np.float64,order='C')
mc3=np.zeros((w,h,3),dtype=np.float64,order='C')

foreground=np.zeros((w,h,3),dtype=np.uint8,order='C')
background=np.zeros((w,h,3),dtype=np.uint8,order='C')
var=np.zeros((w,h,3),dtype=np.float64,order='C')


In [5]:
nf=0
#ret,frame = cap.read()
while(cap.isOpened()):
    ret,frame = cap.read()
    if ret == True:  
        nf=nf+1
    else: 
        break
print("Total Frames in Video",(nf+1))
     

Total Frames in Video 999


## Nomencalture used

##### c1 - blue;
##### c2 - green;
##### c3 - red;
##### ac1,ac2,ac3 - reperesent the ratio of pie j and 2 pie sigma for blue,green and red colour respectively;
##### alpha - learning rate;
##### ρ(rho) = αη(Xt|μk, σk)


In [ ]:
cap = cv2.VideoCapture('umcp.mpg')
for n in range(0,(nf+1)):
    ret,frame = cap.read()    
    for x in range(w):
         for y in range(h):
            #sc1=[0,0,0]; 
            ac1=[0,0,0]       #ac1,ac2,ac3 - reperesent the ratio of pie j and 2*pie*sigma for blue,green and red colour respectively               
            #sc2=[0,0,0]; 
            ac2=[0,0,0]                
            #sc3=[0,0,0]; 
            ac3=[0,0,0]
                    
            match=-1
            zc1=0;zc2=0;zc3=0  #no of matching gaussians for ci colour
            
            alpha = 0.025     # alpha - learning rate
            rho = 0.1         #ρ = αη(Xt|μk, σk)
            
            sum_wc1=0; sum_wc2=0; sum_wc3=0   #Sum of prior probability
            
            if n==0: 
                mc1[x][y][0] = 50;     #initializing the means of 3 Gaussians modelling the blue component (c1) of (x,y) pixel
                mc1[x][y][1] = 100;
                mc1[x][y][2] = 160;
                
                                       
                mc2[x][y][0] = 50;    #initializing the means of 3 Gaussians modelling the green component (c2) of (x,y) pixel
                mc2[x][y][1] = 100;
                mc2[x][y][2] = 160;
                

                mc3[x][y][0] = 50;    #initializing the means of 3 Gaussians modelling the red component (c3) of (x,y) pixel             
                mc3[x][y][1] = 100;
                mc3[x][y][2] = 160;
              
                for i in range(0,3):
                    var[x][y][i] = 15;      #initializing variance of 3 Gaussians (i) modelling (x,y) pixel as 15 (Assumption - Covarainace matrix - diagonal matrix) 
                    wc1[x][y][i] = 0.333;   #wc1 = prior probability of 3 Gaussians(i) modelling blue component (c1) of pixel (x,y)  
                    wc2[x][y][i] = 0.333;
                    wc3[x][y][i] = 0.333;                   
           
            blue=frame[x,y,0]
            green=frame[x,y,1]
            red=frame[x,y,2]
            
            #Checking whether the current pixel matches any one of the Gaussians. 
            #The μ and σ parameters for unmatched distributions remain the same. The parameters of the distribution
            #which matches the new observation are updated as follows
            #            μt = (1 − ρ)μt−1 + ρXt 
            #            σ2t = (1 − ρ)σ2
            #            t−1 + ρ(Xt − μt)T (Xt − μt) 
            #            where
            #               ρ = αη(Xt|μk, σk)
        
            for i in range(0,3):
                if(abs(blue-mc1[x][y][i])<=2.5 * np.sqrt(var[x][y][i])):
                    match=i
                    mc1[x][y][match] = (1 - rho) * mc1[x][y][match] + rho * blue
                    var[x][y][match] = (1 - rho) * var[x][y][match] + rho * (blue - mc1[x][y][match]) * (blue - mc1[x][y][match])
                    wc1[x][y][match] = (1 - alpha) * wc1[x][y][match] + alpha * 1
                    #sc1[match] = 1
                    zc1=zc1+1
                    
                else:
                    wc1[x][y][i] = (1 - alpha) * wc1[x][y][i]
                
                sum_wc1+=wc1[x][y][i]
         
                if(abs(green-mc2[x][y][i])<=2.5* np.sqrt(var[x][y][i])):
                    match=i
                    mc2[x][y][match] = (1 - rho) * mc2[x][y][match] + rho * green
                    var[x][y][match] = (1 - rho) * var[x][y][match] + rho * (green - mc2[x][y][match]) * (green - mc2[x][y][match])
                    wc2[x][y][match] = (1 - alpha) * wc2[x][y][match] + alpha * 1
                    #sc2[match] = 1
                    zc2=zc2+1
                    
                else:
                    wc2[x][y][i] = (1 - alpha) * wc2[x][y][i]
                sum_wc2+=wc2[x][y][i]
          
                if(abs(red-mc3[x][y][i])<=2.5* np.sqrt(var[x][y][i])):
                    match=i
                    mc3[x][y][match] = (1 - rho) * mc3[x][y][match] + rho * red
                    var[x][y][match] = (1 - rho) * var[x][y][match] + rho * (red - mc3[x][y][match]) * (red - mc3[x][y][match])
                    wc3[x][y][match] = (1 - alpha) * wc3[x][y][match] + alpha * 1
                    #sc3[match] = 1
                    zc3=zc3+1
                        
                else:
                    wc3[x][y][i] = (1 - alpha) * wc3[x][y][i]
                sum_wc3+=wc3[x][y][i]
                                        
        # Normalise Prior Probabilities (pi)
        
            for i in range(0,3):
                wc1[x][y][i]=wc1[x][y][i]/sum_wc1
                wc2[x][y][i]=wc2[x][y][i]/sum_wc2
                wc3[x][y][i]=wc3[x][y][i]/sum_wc3
                                   
            for i in range(0,3):
                ac1[i] = wc1[x][y][i] / np.sqrt(var[x][y][i])
                ac2[i] = wc2[x][y][i] / np.sqrt(var[x][y][i])
                ac3[i] = wc3[x][y][i] / np.sqrt(var[x][y][i])                                  
                
       # Arrange in decreasing order of a (the ratio of pie j and 2*pie*sigma)
            
            for i in range(0,3):
                max=i
                for j in range(i+1,3):
                    if (ac1[j] > ac1[max]):
                            max = j
                ac1[i],ac1[max]=ac1[max],ac1[i]                        
                mc1[x][y][i],mc1[x][y][max]=mc1[x][y][max],mc1[x][y][i]
                var[x][y][i],var[x][y][max]=var[x][y][max],var[x][y][i]                        
                wc1[x][y][i],wc1[x][y][max]=wc1[x][y][max],wc1[x][y][i]
                #sc1[i],sc1[max]=sc1[max],sc1[i]
               
            for i in range(0,3):
                max=i
                for j in range(i+1,3):    
                    if (ac2[j] > ac2[max]):
                            max = j
                ac2[i],ac2[max]=ac2[max],ac2[i]                        
                mc2[x][y][i],mc2[x][y][max]=mc2[x][y][max],mc2[x][y][i]
                var[x][y][i],var[x][y][max]=var[x][y][max],var[x][y][i]                        
                wc2[x][y][i],wc2[x][y][max]=wc2[x][y][max],wc2[x][y][i]
                #sc2[i],sc2[max]=sc2[max],sc2[i]                        
                 
            for i in range(0,3):
                max=i
                for j in range(i+1,3):
                    if (ac3[j] > ac3[max]):
                            max = j
                ac3[i],ac3[max]=ac3[max],ac3[i]                        
                mc3[x][y][i],mc3[x][y][max]=mc3[x][y][max],mc3[x][y][i]
                var[x][y][i],var[x][y][max]=var[x][y][max],var[x][y][i]                        
                wc3[x][y][i],wc3[x][y][max]=wc3[x][y][max],wc3[x][y][i]
                #sc3[i],sc3[max]=sc3[max],sc3[i]
                                              
        # Replacing the Least Probable Gaussian with the new one
        
            if zc3==0:
                mc3[x][y][2] = red
                var[x][y][2] = 100000

            if zc2==0:
                mc2[x][y][2] = green
                var[x][y][2] = 100000

            if zc1==0:
                mc1[x][y][2] = blue
                var[x][y][2] = 100000


        # Logic for asssigning Background and Foreground
        
            Bc1=0
            pi_sumc1=0
            for i in range(0,3):                
                if(pi_sumc1>0.8):
                    break
                pi_sumc1+=wc1[x][y][i]
                Bc1=Bc1+1
            
            Bc2=0
            pi_sumc2=0
            for i in range(0,3):
                if(pi_sumc2>0.8):
                    break
                pi_sumc2+=wc2[x][y][i]
                Bc2=Bc2+1
                
            Bc3=0
            pi_sumc3=0
            for i in range(0,3):
                if(pi_sumc3>0.8):
                    break
                pi_sumc3+=wc3[x][y][i]
                Bc3=Bc3+1
                 
            
            if(zc1==0 or abs(blue-mc1[x][y][0])>2.5*np.sqrt(var[x][y][0])):
                foreground[x,y,0]=blue
                background[x,y,0]=mc1[x][y][0]
                
            elif Bc1==1:
                foreground[x,y,0]=255
                background[x,y,0]=blue
                
            else:
                foreground[x,y,0]=255
                background[x,y,0]=mc1[x][y][0]
        
            if(zc2==0 or abs(green-mc2[x][y][0])>2.5*np.sqrt(var[x][y][0])):   
                foreground[x,y,1]=green
                background[x,y,1]=mc2[x][y][0]
            elif Bc2==1:
                foreground[x,y,1]=255
                background[x,y,1]=green
            else:
                foreground[x,y,1]=255
                background[x,y,1]=mc2[x][y][0]
       
            if(zc3==0 or abs(red-mc3[x][y][0])>2.5*np.sqrt(var[x][y][0])):
                foreground[x,y,2]=red
                background[x,y,2]=mc3[x][y][0]
            elif Bc3==1:
                foreground[x,y,2]=255
                background[x,y,2]=red
            else:
                foreground[x,y,2]=255
                background[x,y,2]=mc3[x][y][0]

#Wrting Background and Foreground frames to the disk
                
    cv2.imwrite( r"./Background/%d.jpg" %n, background );
    med = cv2.medianBlur(foreground,3)
    cv2.imwrite( r"./Foreground/%d.jpg" %n, med );
    if cv2.waitKey(1000) == 27 & 0xFF:
        break

### Creating Background Video from Background Frames

In [6]:
import cv2
import numpy as np
import glob
import os


img_array = []

arr = []
for filename in glob.glob('./Background/*.jpg'):
    fn  = os.path.basename(filename)
    # print(fn)
    fn1 = os.path.splitext(fn)[0]
    arr.append(fn1)  

arr.sort(key = int)

for filename in arr:
    img = cv2.imread("./Background/"+filename+".jpg")
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('background.avi',cv2.VideoWriter_fourcc(*'DIVX'), 31, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()



### Creating Foreground Video from Foreground Frames

In [7]:

img_array_f = []

arr_f = []
for filename in glob.glob('./Foreground/*.jpg'):
    fn  = os.path.basename(filename)
    # print(fn)
    fn1 = os.path.splitext(fn)[0]
    arr_f.append(fn1)  

arr_f.sort(key = int)


for filename in arr_f:
    img = cv2.imread("./Foreground/"+filename+".jpg")
    height, width, layers = img.shape
    size = (width,height)
    img_array_f.append(img)
 
 
out = cv2.VideoWriter('foreground.avi',cv2.VideoWriter_fourcc(*'DIVX'), 31, size)
 
for i in range(len(img_array_f)):
    out.write(img_array_f[i])
out.release()